In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, 
from tensorflow.keras.layers import Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Raw Data Loading
df = pd.read_csv('/content/drive/My Drive/[Machine Learning]/data/mnist/train.csv')


##### 결측치와 이상치는 없음 #####


##### 이미지 확인 #####
img_data = df.drop('label', axis=1, inplace=False).values

fig = plt.figure()
fig_arr = []

for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28), 
                      cmap='gray')

plt.tight_layout()
plt.show()


# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', 
                         axis=1, 
                         inplace=False), 
                 df['label'], 
                 test_size=0.3, 
                 random_state=0)
# test_size : test set의 비율 (0.3 => 30%)
# random_state : split할 때 랜덤하게 split하게 되는데 이를 일정하게 고정(seed의 개념)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

##### Tensorflow 2.x implementation #####

model = Sequential()

# Conv2D(필터개수, kernel_size, activation='relu')
model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 activation='relu', 
                 input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 activation='relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))
model.add(Dense(units=256, 
                activation='relu'))
model.add(Dense(units=10, 
                activation='softmax'))

print(model.summary())

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

history = model.fit(x_data_train_norm.reshape(-1,28,28,1),
                    t_data_train,
                    epochs=200,
                    batch_size=100,
                    verbose=1,
                    validation_split=0.3
          )

model.evaluate(x_data_test_norm.reshape(-1,28,28,1), t_data_test)

plt.plot(history.history['sparse_categorical_accuracy'], color='r')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='b')
plt.ylim(0.96,1.0)

plt.show()
